<a href="https://colab.research.google.com/github/arinaaandreeva/ML_lab_CV/blob/main/Lab_CV_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torchvision import transforms

In [3]:
# Вводим данные из json, который получаем на kaggle

# ! pip install -q kaggle
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle
import kagglehub
kagglehub.login()
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
"""Загрузка файлов с Kaggle"""
!kaggle competitions download -c cv-lab-dm-24
!unzip cv-lab-dm-24.zip -d data

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: data/train_images/train_images/1478897974968694375_jpg.rf.o1Ac1ItPbQ6FFJCGY78E.jpg  
  inflating: data/train_images/train_images/1478897976111180782_jpg.rf.bSrAsH2buO80SFXjHxdr.jpg  
  inflating: data/train_images/train_images/1478897976682370183_jpg.rf.ENGgeA9v60NB36BP0knj.jpg  
  inflating: data/train_images/train_images/1478897977253835405_jpg.rf.HAdAOB6nYjjLPbeJ9g0z.jpg  
  inflating: data/train_images/train_images/1478897978395505376_jpg.rf.NKaVdSycFHj3EIJbLAnK.jpg  
  inflating: data/train_images/train_images/1478897978967827296_jpg.rf.KxTNSOndy3LaZTzXv4Ik.jpg  
  inflating: data/train_images/train_images/1478897979538281454_jpg.rf.SPOxwxPjRYfMRPvrdgCx.jpg  
  inflating: data/train_images/train_images/1478897980109393592_jpg.rf.iqHzfLsawvDB17dZ40nZ.jpg  
  inflating: data/train_images/train_images/1478897980682420726_jpg.rf.WoX9VlZS4eNBhOEcOXv1.jpg  
  inflating: data/train_images/train_images/147889

In [61]:
"""
    Файлы COCO
    Их позже подгружают в модель (например, чере torchvision.datasets.CocoDetection)
    В json содржится информация о bounding box, area & category_id
    Категория (category_id) используется как таргет
    """
import json
with open('/content/data/usdc_train.json', 'r') as f:
    data = json.load(f)


# # Извлечение категорий и изображений
# categories = data['categories']  # Список категорий
# images = data['images']  # Список изображений
# annotations = data['annotations']  # Список аннотаций к изображениям

# # Преобразование в DataFrame (если необходимо)
# categories_df = pd.DataFrame(categories)
# images_df = pd.DataFrame(images)
# annotations_df = pd.DataFrame(annotations)


## Задание 1.
### Данные без нормализации

In [7]:
"""Загрузка данных без обработки
    Нужно раскомментировать код, чтобы загрузилось"""
def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'): # у нас файлы jpg
            filepath = os.path.join(folder_path, filename)
            img = cv2.imread(filepath) # пиксели имеют тип uint8 (целое число от 0 до 255)
            if img is not None:
                images.append((filename, img))
    return images


# Загружаем изображения из папок
train_path = "/content/data/train_images/train_images"
test_path = '/content/data/test_images/test_images'
#train_images = load_images(train_path)
#test_images = load_images(test_path)

###Local Contrast Normalization (LCN)

In [8]:
def local_contrast_normalization(img, kernel_size=8):
    """
    Выполняет Local Contrast Normalization (LCN).
    kernel_size: Размер локального окна (например, 8x8).
    """
    # Преобразуем изображение в float32
    img = img.astype(np.float32) / 255.0

    # Вычисляем локальное среднее значение
    mean = cv2.blur(img, (kernel_size, kernel_size))

    # Вычисляем локальное стандартное отклонение
    squared_diff = cv2.blur((img - mean) ** 2, (kernel_size, kernel_size))
    std = np.sqrt(squared_diff+ 1e-5) # добавляем епсилон, чтобы не было деления на 0

    # Применяем нормализацию
    normalized_img = (img - mean) / std

    return normalized_img

"""
    Применяет LCN ко всем изображениях в указанной папке.
    Сохраняет обработанные изображения в указанную выходную папку.
    """

def apply_lcn_to_images(folder_path, output_folder):

    # Проверяем, существует ли выходная папка, если нет — создаем её
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Обрабатываем все изображения в папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):  # Обрабатываем только файлы .jpg
            filepath = os.path.join(folder_path, filename)
            img = cv2.imread(filepath)  # Загружаем изображение (тип uint8)

            if img is not None:
                # Преобразуем в grayscale (если изображение цветное)
                if len(img.shape) == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Применяем Local Contrast Normalization
                normalized_img = local_contrast_normalization(img)

                # Преобразуем изображение обратно в uint8 для сохранения
                normalized_img_uint8 = np.clip(normalized_img * 255, 0, 255).astype(np.uint8)

                # Сохраняем результат в выходную папку с оригинальным именем
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, normalized_img_uint8)

output_folder = '/content/data/lcn_images'  # Папка для сохранения обработанных изображений

apply_lcn_to_images(train_path, output_folder)

In [7]:
"""Через torch, x- изображение"""
# def local_contrast_normalization(x, kernel_size=7):
#     x = x.float()
#     # Локальное среднее
#     mean = F.avg_pool2d(x, kernel_size, stride=1, padding=kernel_size//2)
#     # Локальное стандартное отклонение
#     squared_diff = F.avg_pool2d((x - mean) ** 2, kernel_size, stride=1, padding=kernel_size//2)
#     std = torch.sqrt(squared_diff + 1e-5)  # Добавляем небольшой эпсилон для устойчивости

#     return (x - mean) / std


'Через torch, x- изображение'

### Local response normalization

In [8]:
def local_response_normalization(x, radius=5, alpha=1.0, beta=0.75, k=2.0):
    """
    Применяет Local Response Normalization (LRN) к тензору x.

    :param x: Входной тензор (например, изображение).
    :param radius: Радиус локальной нормализации.
    :param alpha: Коэффициент масштабирования.
    :param beta: Экспоненциальный коэффициент.
    :param k: Смещение для улучшения устойчивости.
    :return: Нормализованный тензор.
    """
    x = x.float()

    # Размер ядра для свертки (с учетом радиуса)
    kernel_size = 2 * radius + 1

    # Создание весов для свертки
    weights = torch.ones(x.size(1), 1, kernel_size, kernel_size).to(x.device)  # Один вес для каждого канала

    # Процесс локальной нормализации (свёртка с квадратами значений)
    square_sum = F.conv2d(x**2, weight=weights, stride=1, padding=radius, groups=x.size(1))

    # Нормализация
    norm = (alpha * square_sum + k) ** beta
    return x / norm

In [ ]:
def normalization_lrn(folder_path, output_folder, radius=5, alpha=1.0, beta=0.75, k=2.0):
    """
    Обрабатывает все изображения в папке и применяет Local Response Normalization (LRN),
    затем сохраняет результат в другую папку.

    :param folder_path: Путь к папке с изображениями.
    :param output_folder: Путь к папке для сохранения обработанных изображений.
    :param radius: Радиус для LRN.
    :param alpha: Коэффициент масштабирования.
    :param beta: Экспоненциальный коэффициент.
    :param k: Смещение для LRN.
    """
    # Создаем папку для вывода, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Трансформатор для преобразования изображения в тензор
    transform = transforms.ToTensor()

    # Обрабатываем все изображения в папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Убедитесь, что работает с изображениями
            filepath = os.path.join(folder_path, filename)

            # Загружаем изображение
            img = Image.open(filepath).convert('RGB')  # Преобразуем в RGB, если изображение не в этом формате

            # Преобразуем изображение в тензор
            x = transform(img).unsqueeze(0)  # Добавляем размерность для batch_size, т.е. (1, 3, H, W)

            # Применяем Local Response Normalization (LRN)
            normalized_x = local_response_normalization(x, radius, alpha, beta, k)

            # Преобразуем тензор обратно в изображение и сохраняем
            normalized_img = normalized_x.squeeze(0).clamp(0, 1).numpy().transpose(1, 2, 0)  # (H, W, C)
            normalized_img = (normalized_img * 255).astype('uint8')  # Преобразуем в диапазон 0-255 для сохранения

            # Сохраняем обработанное изображение
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))  # Сохраняем в BGR

    print(f"Обработка завершена. Изображения сохранены в {output_folder}")

# output_folder = '/content/data/lrn_images'  # Путь к папке для сохранения нормализованных изображений

# normalization_lrn(train_path, output_folder, radius=5, alpha=1.0, beta=0.75, k=2.0)

В современных задачах обнаружения объектов на автопилоте LRN не является оптимальным выбором, так как он устарел.когда объекты плохо видны из-за изменений контраста или освещенности, Local Contrast Normalization (LCN) может быть полезным методом. Это может помочь сделать детали объектов более явными и улучшить их обнаружение в условиях темноты, тумана и т.п.

In [91]:
for i in range(len(data['annotations'])):
  for j in range(2):
    if(data['annotations'][i]['bbox'][j] + data['annotations'][i]['bbox'][j+2] >= 512):
      data['annotations'][i]['bbox'][j+2] -= 3
    if(data['annotations'][i]['bbox'][j] < 0):
      data['annotations'][i]['bbox'][j] = 0

In [ ]:
# поэтому используем LCN и достаем из папки данные
#train_images = load_images('/content/data/lcn_images')

In [9]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [74]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.5, hue=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),#affine
    A.Resize(224, 224),
    #ToTensorV2(),
], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))

In [92]:
i = 0
bboxes = []
for image in load_images('/content/data/lcn_images'):
  if not os.path.exists("/content/data/augmented_images"):
    os.makedirs("/content/data/augmented_images")
  transformed = transform(image=image[1], bboxes=[data['annotations'][i]['bbox']], class_labels=[data['categories'][data['annotations'][i]['category_id']]['name']])
  cv2.imwrite(f'/content/data/augmented_images/{i}.jpg', transformed["image"])
  bboxes.append(transformed["bboxes"])
  i += 1
